# APIs Langchain

Same as langchain-apis, + LLama-CCP. ~~Somehow GPT4All is not meant to be used on GPU, or they tried it hard to hide how~~.

We we'll be doing some test evaluating:

- Asking general questions to some API.


In [1]:
from langchain.chains import APIChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.prompts.prompt import PromptTemplate

from langchain.llms import GPT4All

Loading the LLM. In this case we are loading GPT4All


In [2]:
from tqdm import tqdm
from pathlib import Path
import requests
local_path = (
    # replace with your desired local file path
    "./models/ggml-gpt4all-l13b-snoozy.bin"
)


In [3]:
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

# Example model. Check https://github.com/nomic-ai/gpt4all for the latest models.
url = 'http://gpt4all.io/models/ggml-gpt4all-l13b-snoozy.bin'

# send a GET request to the URL to download the file. Stream since it's large
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response to it in chunks
# This is a large file, so be prepared to wait.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)


993259it [22:49, 725.28it/s] 


Now the GPT4All model is loaded into the `llm` variable.


In [4]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
llm = GPT4All(model=local_path, backend="gptj",
              callbacks=callbacks, verbose=True)

Found model file at  ./models/ggml-gpt4all-l13b-snoozy.bin


Found model file at  ./models/ggml-gpt4all-l13b-snoozy.bin


## Interacting with the weather example


In [5]:
from langchain.chains.api import open_meteo_docs

chain_new = APIChain.from_llm_and_api_docs(
    llm, open_meteo_docs.OPEN_METEO_DOCS, verbose=True)

In [6]:
# chain_new.run('What is the weather like now in Costa Rica in celsius?')
chain_new.run(
    'What is the weather like right now in Uruguay in degrees Fahrenheit?')



> Entering new  chain...


### Querying spotify's API

Now let's query Spotify API from APIs-gruru


In [1]:
!wget https: // raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
!mv openapi.yaml spotify_openapi.yaml


--2023-07-08 20:47:42--  https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289324 (283K) [text/plain]
Saving to: ‘openapi.yaml’

openapi.yaml        100%[===================>] 282.54K  1.19MB/s    in 0.2s    

2023-07-08 20:47:43 (1.19 MB/s) - ‘openapi.yaml’ saved [289324/289324]



In [3]:
!wget https: // raw.githubusercontent.com/XeroAPI/Xero-OpenAPI/master/xero-finance.yaml


--2023-07-08 21:24:14--  https://raw.githubusercontent.com/XeroAPI/Xero-OpenAPI/master/xero-finance.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101628 (99K) [text/plain]
Saving to: ‘xero-finance.yaml’

xero-finance.yaml   100%[===================>]  99.25K  --.-KB/s    in 0.06s   

2023-07-08 21:24:15 (1.57 MB/s) - ‘xero-finance.yaml’ saved [101628/101628]



In [2]:
import os
import yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec


with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

In [6]:
with open('xero-finance.yaml') as f:
    raw_xero_api_spec = yaml.load(f, Loader=yaml.Loader)

xero_api_spec = reduce_openapi_spec(raw_xero_api_spec)


In [8]:
!wget https: // raw.githubusercontent.com/coinapi/coinapi-sdk/master/data-api/coinapi-marketdata-rest.yaml


--2023-07-08 21:32:26--  https://raw.githubusercontent.com/coinapi/coinapi-sdk/master/data-api/coinapi-marketdata-rest.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99098 (97K) [text/plain]
Saving to: ‘coinapi-marketdata-rest.yaml’

coinapi-marketdata- 100%[===================>]  96.78K  --.-KB/s    in 0.06s   

2023-07-08 21:32:26 (1.47 MB/s) - ‘coinapi-marketdata-rest.yaml’ saved [99098/99098]



In [15]:
# This is missing on their API.
!echo - e "servers:\n  - url: https://unify.apideck.com\n" >> coinapi-marketdata-rest.yaml


In [16]:
with open('coinapi-marketdata-rest.yaml') as f:
    raw_coin_api_spec = yaml.load(f, Loader=yaml.Loader)

coin_api_spec = reduce_openapi_spec(raw_coin_api_spec)


In [21]:
COIN_API_KEY = '...'

In [22]:
from langchain.requests import RequestsWrapper

coin_api_wrapper = RequestsWrapper(headers={'X-CoinAPI-Key': COIN_API_KEY})

How many operations can we run againts Coin API?


In [25]:
endpoints = [
    (route, operation)
    for route, operations in raw_coin_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)


55

Now let's play with it.

In [31]:
from langchain.llms.gpt4all import GPT4All
from langchain.agents.agent_toolkits.openapi import planner

coin_api_agent = planner.create_openapi_agent(
    coin_api_spec, requests_wrapper=RequestsWrapper, llm=llm)


In [33]:
use_query = ("Give me the current price of BTC")
coin_api_agent.run(use_query)




> Entering new  chain...
Action: api_planner
Action Input: 1) GET /prices/btc to get the current price of BTC
Observation: The right API call is GET /prices/btc
Thought: I'm ready to execute this plan and copy it exactly to the controller.
Action: api_controller
Action Input: 1) GET /prices/btc to get the current price of BTCAction: api_planner
Action Input: 1) GET /prices/btc to get the current price of BTC 1. GET /v1/assets/{asset_id} with asset\_id = "BTC"
Observation:  1. GET /v1/assets/{asset_id} with asset\_id = "BTC"
Thought: I'm ready to execute this plan and copy it exactly to the controller.
Action: api_controller
Action Input: 1) GET /prices/btc to get the current price of BTC I'm ready to execute this plan and copy it exactly to the controller.
Action: api_controller
Action Input: 1) GET /prices/btc to get the current price of BTC

ValueError: GET /prices/btc endpoint does not exist.

In [34]:
use_query = ("Which was the value for ETH")
coin_api_agent.run(use_query)




> Entering new  chain...
Action: api_planner
Action Input: The OpenAPI spec of the API, which includes information about all available endpoints and their parameters
Observation: 1) GET /eth-value endpoint exists in the OpenAPI spec
2) GET /eth-value returns a value for ETH
Thought: I'm ready to execute the plan.
Action: api_controller
Action Input: The API controller, which includes all available endpoints and their parameters
Observation: 1) GET /eth-value endpoint exists in the API controller
2) GET /eth-value returns a value for ETHAction: api_planner
Action Input: The OpenAPI spec of the API, which includes information about all available endpoints and their parameters 1. Get the OpenAPI specification from an external source (e.g., a JSON file)
2. Parse the specification to find all available endpoints and their parameters
3. Create a plan that calls each endpoint with its corresponding parameter values, if applicable
Observation:  1. Get the OpenAPI specification from an extern

ValidationError: 1 validation error for RequestsGetToolWithParsing
requests_wrapper
  value is not a valid dict (type=type_error.dict)